[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/PO_II/blob/main/3_Otimização_Redes/Transportes/transportes.ipynb)

## **Pesquisa Operacional II**

**Prof. Diogo Ferreira de Lima Silva (TEP-UFF)**

Monitores: 
- Rodrigo Celso de Lima Porto (2022)


# PROBLEMA DE TRANSPORTE

Produtos precisam ser enviados a partir de um conjunto de $m$ origens para um conjunto de $n$ destinos. 

- Cada origem $i$ pode fornecer, no máximo $a_i$ unidades. 
- Cada destino $j$ possui uma demanda por $b_j$ unidades.

Sabe-se o custo unitário do envio de produtos da fábrica $i$ para o cliente $j$, dado por $c_{ij}$.

**Objetivo**: minimizar o custo total do transporte. 


O modelo abstrato de programação linear para este tipo de problema é dado como:

$$
\text{Min } z = \sum_{i=1}^{m} \sum_{j=1}^{n} c_{ij}x_{ij}\\
$$
$$
\begin{equation}
  \begin{array}{rlll}
    \text{Sujeito a: } & & \\
    & \sum_{j=1}^{n} x_{ij} \le a_{i} & , \forall \ i=1,...,m & (1) \\
    & \sum_{i=1}^{m} x_{ij} \ge b_{j} & , \forall \ j=1,...,n & (2) \\
    & x_{ij} \ge 0 & , \forall i = 1,...,m & e & j=1,...,n & (3)
  \end{array}
\end{equation}
$$


Onde:

- Cada $x_{ij}$ é uma variável de decisão, representando a quantidade de produtos enviadas do ofertante $i$ para o cliente $j$ 
- $c_{ij}$ contém o custo de transporte da unidade da fábrica $i$ para o cliente $j$ 
- $a_i$ contém a capacidade máxima de cada fábrica $i$;
- $b_j$ contém a demanda de cada cliente $j$;
- As restrições do tipo $(1)$, restrições de capacidade, garantem que a quantidade de produtos despachadas por cada ofertante não seja maior do que sua respectiva oferta;
- As restrições do tipo $(2)$, de demanda, garantem que os clientes tenham as demandas atendidas;
- As restrições do tipo $(3)$ são de não negatividade.


# Vamos modelar uma instância Problema dos Transportes

Uma empresa possui três fábricas que produzem carrinhos de bebê que devem ser remetidos para quatro centros de distribuição. As Fábricas F1, F2 e F3 produzem, respectivamente, 12, 17 e 11 remessas por mês. Cada centro de distribuição CD precisa receber dez remessas por mês. Com base na tabela de custos abaixo, que mostra o custo unitário de transporte das Fábricas à cada CD, quanto deve ser remetido de cada fábrica para cada um dos centros de distribuição para minimizar o custo total de transporte?

<center>

|Fábrica|$CD1$|$CD2$|$CD3$|$CD4$|
|:-----:|:---:|:---:|:---:|:---:|
|$F1$   |40100|65100|20100|35100|
|$F2$   |55100|70100|30100|50100|
|$F3$   |30100|60100|40100|45100|

</center>


## Formulação do Problema

A formulação deste modelo seria uma instância do problema de transporte, podendo ser dado como:

$$
\text{Min } z = 40100x_{11} + 65100x_{12} + 20100x_{13} + 35100x_{14} + 55100x_{21} + 70100x_{22} + 30100x_{23} + 50100x_{24} + 30100x_{31} + 60100x_{32} + 40100x_{33} + 45100x_{34}\\
$$
$$
\begin{equation}
  \begin{array}{rll}
    \text{Sujeito a: } & & \\
    & x_{11} + x_{12} + x_{13} + x_{14} \le 12 & \text{(Fábrica 1)} \\
    & x_{21} + x_{22} + x_{23} + x_{24} \le 17 & \text{(Fábrica 2)} \\
    & x_{31} + x_{32} + x_{33} + x_{34} \le 11 & \text{(Fábrica 3)} \\
    & x_{11} + x_{21} + x_{31} \ge 10 & \text{(Centro de Distribuição 1)} \\
    & x_{12} + x_{22} + x_{32} \ge 10 & \text{(Centro de Distribuição 2)} \\
    & x_{13} + x_{23} + x_{33} \ge 10 & \text{(Centro de Distribuição 3)} \\
    & x_{14} + x_{24} + x_{34} \ge 10 & \text{(Centro de Distribuição 4)} \\
    & x_{ij} \ge 0 \\
  \end{array}
\end{equation}
$$
Onde $x_{ij}$ é a quantidade de remessas enviadas da fábrica $i$ para o centro de distribuição $j$.

In [ ]:
#Execute esse bloco caso esteja executando no Google Colab
!pip install -q pyomo
!pip install -i https://pypi.gurobi.com gurobipy

---
# Resolução
---

<p align=justify> &emsp; Vamos imaginar que os custos de transporte das fábricas até os clientes estejam organizados em uma matriz e as ofertas de remessa de cada fábrica e demandas de cada centro de distribuição estejam armazenados em duas listas distintas, como mostrado a seguir:</p>

In [120]:
import numpy as np
import pyomo.environ as pyo # Importando o Pyomo
import pandas as pd


In [121]:
c = pd.DataFrame ([[40100, 65100, 20100, 35100], # F1
                    [55100, 70100, 30100, 50100], # F2
                    [30100, 60100, 40100, 45100]])

c_dict = c.stack().to_dict()

# Vamos armazenar nossas ofertas e demandas nas listas abaixo:
a = [12, 17, 11]
b = [10, 10, 10, 10]

m = len(a)
n = len(b)

In [122]:
# Criando uma instância do modelo
M = pyo.ConcreteModel() 

In [123]:
# Criando dois índices para serem usados no pyomo
M.fabricas = pyo.Set(initialize = range(m)) # Índice para as fábricas
M.cds = pyo.Set(initialize = range(n)) # Índice para os centros de distribuição

In [124]:
# PASSANDO OS PARÂMETROS DO PROBLEMA

# Custos de transporte da fábrica i para o centro de dsitribuição j
M.custos = pyo.Param(M.fabricas, M.cds, initialize = c_dict)

# Capacidade de cada fábrica
M.capacidades = pyo.Param (M.fabricas, initialize = a)

# Demanda de cada centro de distribuição
M.demandas = pyo.Param (M.cds, initialize = b)
  

In [125]:
# VARIÁVEIS DE DECISÃO
# xij representa a quantidade de remessas enviada da fábrica i para o CD j
M.x = pyo.Var(M.fabricas, M.cds, within= pyo.NonNegativeReals)


In [126]:
def custo_total (M):
    return sum(M.x[i,j] * M.custos[i,j] for i in M.fabricas for j in M.cds)

# FUNÇÃO OBJETIVO
M.obj = pyo.Objective(rule= custo_total ,sense= pyo.minimize)
  

In [127]:
# R1: O total de remessas despachada por cada fábrica não pode ser maior que a sua oferta.
M.R_ofertas = pyo.Constraint(M.fabricas, rule= lambda M, i: sum(M.x[i, j] for j in M.cds) <= M.capacidades[i])

# R2: Cada centro de distribuição deve ter a sua demanda atendida
M.R_demandas = pyo.Constraint(M.cds, rule= lambda M, j: sum(M.x[i, j] for i in M.fabricas) >= M.demandas[j])

In [128]:
# RESOLUÇÃO DO MODELO
pyo.SolverFactory('glpk').solve(M)

{'Problem': [{'Name': 'unknown', 'Lower bound': 1624000.0, 'Upper bound': 1624000.0, 'Number of objectives': 1, 'Number of constraints': 8, 'Number of variables': 13, 'Number of nonzeros': 25, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.07978487014770508}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [129]:
print(f"z= {pyo.value(M.obj)}")

for k in M.x.keys(): 
    if pyo.value(M.x[k]) > 0:
        print (f"x_{k} = {pyo.value(M.x[k])}")

z= 1624000.0
x_(0, 2) = 2.0
x_(0, 3) = 10.0
x_(1, 1) = 9.0
x_(1, 2) = 8.0
x_(2, 0) = 10.0
x_(2, 1) = 1.0


---
---
# EXERCÍCIO DE FIXAÇÃO
---
---

> <p align=justify>Uma companhia distribuidora de bebidas tem 2 centros de produção - Araraquara e São José dos Campos - e 3 mercados consumidores principais - São Paulo, Belo Horizonte e Rio de Janeiro. O custo unitário de se transportar uma unidade do produto de cada centro de produção a cada mercado consumidor é dada na tabela a seguir.Nessa tabela, também são apresentadas as demandas de cada mercado e a quantidade máxima disponível do produto em cada centro de produção no próximo período.</p>

<center>
<table>
  <caption>
    <b>Custos unitários dos centros de produção aos mercados consumidores</b>
  </caption>
  <tr align=center>
    <th rowspan=2>Centros de produção</th>
    <th colspan=3>Mercados consumidores</th>
    <th rowspan=2>Suprimento disponível</th>
  </tr>
  <tr align=center>
    <th>São Paulo (3)</th>
    <th>Belo Horizonte (4)</th>
    <th>Rio de Janeiro (5)</th>
  </tr>
  <tr align=center>
    <td>Araraquara (1)</td>
    <td>4</td>
    <td>2</td>
    <td>5</td>
    <td>800</td>
  </tr>
  <tr align=center>
    <td>S. J. Campos (2)</td>
    <td>11</td>
    <td>7</td>
    <td>4</td>
    <td>1000</td>
  </tr>
  <tr align=center>
    <td>Demanda dos mercados</td>
    <td>500</td>
    <td>400</td>
    <td>900</td>
  </tr>
</table>
</center>

---
Fonte: ARENALES, Marcos; ARMENTANO, Vinícius; MORANITO, Reinaldo e YANASSE, Horacio. <b>Pesquisa Operacional para Cursos de Engenharia</b>. 2ª edição. Exemplo 2.3, pág. 22.

---

### (a) Resolva pelo Pyomo

In [ ]:
import pyomo.environ as pyo

